In [10]:
# from src.motion_control.LocalNavigator import LocalNavigator
# from src.motion_control.MotionControl import MotionControl
# from src.motion_control.ThymioController import ThymioController
from src.vision.vision import ComputerVision
from src.global_navigation.global_navigation import AStarNavigation


vision = ComputerVision()
image_var = vision.get_map()

global_navigation = AStarNavigation(safety=5, file_path=None)
global_path = global_navigation.run_with_var(image_var)

print(global_path)

# #Global navigation 
# path = [(19, 1), (20, 1), (21, 1), (21, 2), (21, 3), (21, 4)]
# 
# 
# #Motion control, filtering, local navigation
# thymio = ThymioController()
# thymio.connect(timeout=5)
# local_navigator = LocalNavigator(thymio)
# motion_control = MotionControl(thymio, path)



AttributeError: module 'cv2' has no attribute 'aruco'

In [9]:
try:
    # Define path (list of waypoints)
    while True:

        if local_navigator.obstacle_detected():
            thymio.stop()
            local_navigator.handle_obstacle()
            
        if local_navigator.kidnapping_detect():
            thymio.stop()
            
        motion_control.execute_path()

finally:
    thymio.disconnect()

NameError: name 'thymio' is not defined